In [12]:
%load_ext autoreload
%autoreload 2
from src.dataset import Dataset
import matplotlib.pyplot as plt

plt.style.use(['ggplot', 'https://raw.githubusercontent.com/AlanBlanchet/matplotlib_styles/master/vscode_blue.mplstyle'])

Dataset.init()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


src.dataset.dataset.Dataset

In [ ]:
Dataset.example("topics1.csv", random_state=0, version=2, interactive=True)

--------------- index=1
Original ============================================
 Is it okay to put your Google Maps API Key into your source code and publish it? 
 Others could take it and misuse it, but I don't want every developer / user to get their own API key and type it in somewhere. If the owner of the key is responsible, should I create a new google account for the project? (The project is a desktop application in Objective-C and a small developer tool.) 
 What would be the best way to make this convenient? 

Parsed ==============================================
 okay put googl map api key sourc code publish other could take misus n t want everi develop user get api key type somewher owner key respons creat new googl account project project desktop applic object c small develop tool would best way make conveni
Targets ========== security|api|open-source|google-maps|publish
None


Voici les types d'approches à tester :

- Bag of Words (BoW) pour nous
- Word Embeddings : Word2Vec

## Bag of Words

Dans cet partie on va effectuer un bag of words avec tous les mots que l'on a.

In [10]:
df = Dataset.use("topics1.csv", version=2)
df.head()

,title,text,target
0,Emacs ido-style shell,command line shell shell custom support emac s...,bash|shell|emacs|eshell|ido
1,Include Google Maps API Key in open source pro...,okay put googl map api key sourc code publish ...,security|api|open-source|google-maps|publish
2,How do I do large non-blocking updates in Post...,want larg updat tabl postgresql n t need trans...,postgresql|transactions|sql-update|plpgsql|dblink
3,How can hibernate access a private field?,hibern access privat field method java class e...,java|hibernate|reflection|field|private
4,How to add NSViewController to a responder chain?,m hard time understand find info make nsviewco...,objective-c|macos|cocoa|nsviewcontroller|nsres...
